## Grab long, HQ files from local SQLite DB

In [ ]:
!pip install sqlite3, quickumls
# I needed ~16GB RAM for this step, contributed to GitHub issue fix
!python -m quickumls.install "/mnt/isilon/wang_lab/shared/datasets/UMLS_unzipped/2019AB-full/UMLS_lib/2019AB/META/" quickumlsdata
!python -m pip install git+https://github.com/GeneDx/phenopy.git@develop
!python -m pip install git+https://github.com/GeneDx/txt2hpo.git@develop

In [3]:
!pip install cx_Oracle

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 814 kB 1.2 MB/s eta 0:00:01
You should consider upgrading via the '/cm/shared/apps_chop/python/3.9.1/bin/python3.9 -m pip install --upgrade pip' command.


In [10]:
import os, argparse, sys
import cx_Oracle                                                                       
import time
import sys
import sqlite3
import os
import numpy as np

## Fill in your username and password
clarity_user = ''
clarity_psw = ''

## if 'exaorat2' does not work, try 'exaorav2'
dsnStr = cx_Oracle.makedsn("exaorat2-scan.chop.edu", "1521", service_name="CLTprd_pri.chop.edu")
remote_con = cx_Oracle.connect(user = clarity_user, password = clarity_psw, dsn = dsnStr)
remote_cur = remote_con.cursor()

In [12]:
remote_cmd = """
select hno_info.pat_id,  
       htext.note_id, htext.CONTACT_DATE, pat.birth_date, hno_info.current_author_id
               
        from hno_info
        inner join hno_note_text htext on htext.note_id = hno_info.note_id
        inner join patient pat on pat.pat_id = hno_info.pat_id where rownum <=10
        and hno_info.current_author_id is not NULL
"""

### execute the sql command
remote_cur.execute(remote_cmd)

### fetch all the data 
data = remote_cur.fetchall()

### num of queried patients (as on 12/14/2020)
len(data)

10

In [2]:
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day
print(to_integer(data[1][2]))
print(data[1])

NameError: name 'data' is not defined

In [15]:
with open("/mnt/isilon/wang_lab/jim/patientnotes.tsv", "w") as file:
    print("patient\tnote\tadmission_date\tbirth_date", file=file)
    for entry in data:
        try:
            adate=str(to_integer(entry[2]))
        except AttributeError:
            adate="NA"
        try:
            bdate=str(to_integer(entry[3]))
        except AttributeError:
            bdate="NA"
        print("\t".join([entry[0],entry[1],adate,bdate]),file=file)


In [2]:
import sqlite3
## local sql-db name to store the notes of ASD patients
local_db_name = 'ds_patient_check.db'
## sqlite3.connect create a new sql-db if not exists, and it need ' .commit()' at the end to safe the sql-db
local_con = sqlite3.connect(local_db_name)
## .cursor() is to execute a sql_cmd, and to fetch data.
local_cur = local_con.cursor()
query="""
select * from note_text where length(note_text)>2000 and note_text like '%MRN:%' 
""" # add limit 20 to very end of query to limit and test lines
local_cur.execute(query)
data = local_cur.fetchall()
local_cur.close()


In [3]:
len(data) # 369,598 was Mengge's initial count of DS notes at >2000 chars

124104

## Send notes to txt2hpo

In [4]:
from txt2hpo.extract import Extractor
from timeit import default_timer as timer
import json
extract = Extractor(remove_overlapping=False)

2021-03-08 16:38:20,569 - phenopy - INFO - checking if config file exists: /home/havrillaj/.phenopy
2021-03-08 16:38:20,572 - phenopy - INFO - phenopy 0.5.2
2021-03-08 16:38:20,576 - phenopy - INFO - Using configuration file: /home/havrillaj/.phenopy/phenopy.ini


In [1]:
start = timer()
result = {}
for note in data[:5]:
    result[note[0]] = json.loads(extract.hpo(note[4]).json)
end = timer()
print(end - start)

NameError: name 'timer' is not defined

In [46]:
# Save a dictionary into a json
with open('data.json', 'w') as f:
    json.dump(result, f)

In [47]:
with open('data.json') as json_file:
    dataload = json.load(json_file)